In [1]:
#-------------사용할 패키지-------------#
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import csv
import os
import re
import requests
import pandas as pd

In [2]:
#-------------KOSPI200.csv 불러오기-------------#
code_group=pd.read_csv('KOSPI200.csv',header=None,encoding='utf-8')
code_group

,0,1
0,5930,삼성전자
1,660,SK하이닉스
2,35420,NAVER
3,51910,LG화학
4,5380,현대차
...,...,...
196,5610,SPC삼립
197,145990,삼양사
198,5180,빙그레
199,9410,태영건설


In [3]:
#-------------코드가 6자리이므로 모양 바꾸기-------------#
for i in range(0,201,1):
    code_group.iloc[i,0]='{0:06d}'.format(code_group.iloc[i,0])

code_group

,0,1
0,005930,삼성전자
1,000660,SK하이닉스
2,035420,NAVER
3,051910,LG화학
4,005380,현대차
...,...,...
196,005610,SPC삼립
197,145990,삼양사
198,005180,빙그레
199,009410,태영건설


In [4]:
#-------------Url 기본설정-------------#
BaseUrl="https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}"

In [5]:
#-------------1개 추출 연습-------------#
group="SKC"
code="011790"
page="1"

headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
# User-Agent가 없으면 봇으로 취급당해 naver.finance에서 크롤링이 안된다.
url=BaseUrl.format(code,page)
r=requests.get(url,headers=headers)
soup=BeautifulSoup(r.text,'lxml')

items=soup.find_all('span')
del items[0] # 일별시세 부분 제거(맨 앞 불필요한 부분)

Date=items[0].text
Closing=items[1].text
Start=items[3].text
Max=items[4].text
Min=items[5].text
exchange=items[6].text
            
data=group,Date,Start,Min,Max,Closing,exchange
data # 회사, 날짜, 시가, 저가, 고가, 종가, 거래량

('SKC', '2021.03.19', '0', '0', '0', '114,000', '0')

In [6]:
#-------------실전-------------#
if os.path.exists('Crawling_data.csv'):
    os.remove('Crawling_data.csv') 
# KOSPI200.csv란 파일이 이미 존재하면 지우고 다시 만들기

before100time=datetime.today()-timedelta(100) # 100일 전 날짜 구하기
before99time=datetime.today()-timedelta(99) # 100일 전이 주말인 경우 대비1
before98time=datetime.today()-timedelta(98) # 100일 전이 주말인 경우 대비2

limit_time1=before100time.strftime("%Y.%m.%d")
limit_time2=before99time.strftime("%Y.%m.%d")
limit_time3=before98time.strftime("%Y.%m.%d")
# 문자열 치환

headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

for i in range(0,201,1): # 0~200개 기업 코드 반복문
    group=code_group.iloc[i,1]
    code=code_group.iloc[i,0]
    
    breaker=False # 100일 전까지 데이터가 없을 시 종료를 위한 스위치
    
    for j in range(1,10,1): # 주말에도 했다는 가정 시 최대 페이지(10X10)
        
        try: # 10페이지 전에 종료 시 오류 발생 --> try,except 구문으로 처리
            if breaker:
                break
                
            url=BaseUrl.format(code,j)
            r=requests.get(url,headers=headers)
            soup=BeautifulSoup(r.text,'lxml')
        
            items=soup.find_all('span')
            del items[0]
              
            for k in range(0,10,1): # 한 페이지 당 10개의 날짜가 존재
                
                num=7*k
                
                Date=items[num].text
                Closing=items[num+1].text
                Start=items[num+3].text
                Max=items[num+4].text
                Min=items[num+5].text
                exchange=items[num+6].text
            
                data=group,Date,Start,Min,Max,Closing,exchange
            
                with open('Crawling_data.csv','a',newline='',encoding='utf-8') as f:
                    writer=csv.writer(f)
                    writer.writerow(data)
            
                if Date==limit_time1 or Date==limit_time2 or Date==limit_time3 :
                    breaker=True
                    break
        except:
            break